# Calcul du Internal Product
Le but de ce carnet jupyter est de faire le calcul du produit scalaire
$$
\braket{x|\psi}
$$

avec
$$
\ket{\psi}=P_{\text{G}}P_{\text{J}}\ket{\phi_\text{Pfaff}},
$$
afin de pouvoir calculer la probabilité d'une update. Ce carnet va aussi contenir des exemples d'utilisation des _fast-updates_.

Pour rouler le code présent dans ce carnet, il faut avoir compilé la librairie avec la _feature_ `python-interface`, puis avoir exposé l'interface à l'environnement python à l'aide de `maturin`. Le script `build.sh` présent dans
ce répertoire permet la création d'un environnement virtuel et l'installation de la librairie `impurity`.

In [11]:
import impurity as imp
NSITES = 8
spin_up = 101
spin_down = 164
new_spin_up = 165

## Mise en situation
Pour l'entièreté de ce carnet, le problème qui sera considéré sera une chaine de $8$ spins ayant une connectivité périodique. L'état qui sera considéré sera une configuration $s_{\uparrow}=[01100101]=101$ pour les spins _up_ ainsi qu'une configuration $s_{\downarrow}=[10100100]=164$ pour les spins _down_.

Ajouter une figure

Pour les fast-updates, le _hopping_ sera de spin up et passera de l'indice $1$ à l'indice $0$. La nouvelle configuration _up_ sera donc $s_{\uparrow}=[10100101]=165$.

Ajouter une figure

## Calcul du Projecteur Gutzwiller $P_{\text{G}}$
On utilise la définition du projecteur de Gutzwiller

$$
P_{\text{G}}=e^{\sum_i g_in_{i\uparrow}n_{i\downarrow}},
$$
ou sous sa forme plus commode numériquement
$$
\ln P_{\text{G}}=\sum_i g_i n_{i\uparrow}n_{i\downarrow}.
$$
La somme sur $i$ représente la somme sur l'entièreté des sites. Les $g_i$ sont les paramètres variationnels du projecteur. Nous fixons ces paramètres aux valeurs:

$$
\mathbf{g}=\begin{pmatrix}
-0.23 &-0.59 & -0.58 & -0.8 & -0.57 & -0.47 & 0.4 & -0.62
\end{pmatrix}
$$

### Calcul explicite
Les seuls termes de la somme qui contribuent sont ceux pour lesquels le site contient un spin _up_ et un spin _down_, donc équivalent au produit matriciel suivant. Les indices à l'intérieur du vecteur colonne sont donnés par l'opération
_et bitwise_ entre $s_\uparrow$ et $s_\downarrow$, $\zeta=s_\uparrow\&s_\downarrow$.

$$
\begin{align}
\sum_i g_i n_{i\uparrow}n_{i\downarrow}&=\begin{pmatrix}
-0.23 &-0.59 & -0.58 & -0.8 & -0.57 & -0.47 & 0.4 & -0.62
\end{pmatrix}
\begin{pmatrix}
0 \\ 0 \\ 1 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0
\end{pmatrix}\\
&=-0.58 - 0.47=-1.05
\end{align}
$$

Pour après le _hopping_, les termes qui contribuent change, soit

$$
\begin{align}
\sum_i g_i n_{i\uparrow}n_{i\downarrow}&=\begin{pmatrix}
-0.23 &-0.59 & -0.58 & -0.8 & -0.57 & -0.47 & 0.4 & -0.62
\end{pmatrix}
\begin{pmatrix}
1 \\ 0 \\ 1 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0
\end{pmatrix}\\
&=-0.23-0.58 - 0.47=-1.28
\end{align}
$$

### Calcul numérique

In [2]:
# Calcul pour la première configuration
gutzwiller_parameters = [-0.23, -0.59, -0.58, -0.8, -0.57, -0.47, 0.4, -0.62]
gutzwiller_coefficient = imp.gutzwiller_exponent(spin_up, spin_down, gutzwiller_parameters, NSITES)
print(gutzwiller_coefficient)

# Calcul après le hopping
gutzwiller_coefficient = imp.gutzwiller_fastupdate(
    gutzwiller_coefficient, gutzwiller_parameters, spin_up, spin_down, 1, 0, True
)
print(gutzwiller_coefficient)

-1.0499999999999998
-1.2799999999999998


## Calcul du Projecteur Jastrow $P_{\text{J}}$
On utilise la définition du projecteur de Jastrow

$$
\begin{align}
P_{\text{J}}&=e^{\frac12 \sum_{i\neq j} v_{ij}(n_i-1)(n_j -1)}\\
\ln P_{\text{J}}&=\frac12 \sum_{i\neq j} v_{ij}(n_i-1)(n_j -1)
\end{align}
$$
où les $v_{ij}$ sont les paramètres variationnels, tandis que les opérateurs $n_i$ sont les opérateurs nombre d'occupation sans égars au spin.

$$
\mathbf{v}=\begin{pmatrix}
0 & -0.7 & 0.28 & -0.42 & 0.31 & 0.63 & -0.33 & 1\\
-0.75 & 0 & -0.11 & 0.38 & 0.06 & 0.44 & 0.93 & -0.32\\
-0.49 & -0.67 & 0 & -0.07 & 0.26 & 0.99 & 0 & -0.94\\
0.46 & -0.59 & 0.84 & 0 & -0.13 & 0.33 & -0.98 & 0.96\\
0.25 & -0.26 & -0.6 & 0.77 & 0 & -0.92 & 0.65 & -0.58\\
-0.85 & 0.19 & 0.06 & -0.19 & -0.54 & 0 & 0.01 & -0.3 \\
0.52 & -0.01 & -0.55 & 0.26 & 0.44 & -0.26 & 0 & 0.08\\
-0.32 & -0.05 & 0.01 & -0.25 & 0.25 & -0.46 & 0.5 & 0
\end{pmatrix}
$$

La somme s'écrit comme le produit matriciel suivant, en utilisant le vecteur $\zeta=s_\uparrow+s_\downarrow-\mathbf{1}$

$$
\begin{align}
\frac12 \sum_{i\neq j} v_{ij}(n_i-1)(n_j -1)=\\
\frac12\begin{pmatrix}
0 & 0 & 1 & -1 & -1 & 1 & -1 & 0
\end{pmatrix}
\begin{pmatrix}
0 & -0.7 & 0.28 & -0.42 & 0.31 & 0.63 & -0.33 & 1\\
-0.75 & 0 & -0.11 & 0.38 & 0.06 & 0.44 & 0.93 & -0.32\\
-0.49 & -0.67 & 0 & -0.07 & 0.26 & 0.99 & 0 & -0.94\\
0.46 & -0.59 & 0.84 & 0 & -0.13 & 0.33 & -0.98 & 0.96\\
0.25 & -0.26 & -0.6 & 0.77 & 0 & -0.92 & 0.65 & -0.58\\
-0.85 & 0.19 & 0.06 & -0.19 & -0.54 & 0 & 0.01 & -0.3 \\
0.52 & -0.01 & -0.55 & 0.26 & 0.44 & -0.26 & 0 & 0.08\\
-0.32 & -0.05 & 0.01 & -0.25 & 0.25 & -0.46 & 0.5 & 0
\end{pmatrix}
\begin{pmatrix}
0 \\ 0 \\ 1 \\ -1 \\ -1 \\ 1 \\ -1 \\ 0
\end{pmatrix}\\
=\frac12
\begin{pmatrix}
0 & 0 & 1 & -1 & -1 & 1 & -1 & 0
\end{pmatrix}
\begin{pmatrix}
1.35\\-1.04\\0.8\\2.28\\-2.94\\0.78\\-1.51\\-0.95
\end{pmatrix}\\
=1.875
\end{align}
$$

On peut faire la même chose pour après le hopping, avec $\zeta=[1,-1,1,-1,-1,1,-1,0]$.
$$
\begin{align}
\frac12 \sum_{i\neq j} v_{ij}(n_i-1)(n_j -1)=\\
\frac12\begin{pmatrix}
1 & -1 & 1 & -1 & -1 & 1 & -1 & 0
\end{pmatrix}
\begin{pmatrix}
0 & -0.7 & 0.28 & -0.42 & 0.31 & 0.63 & -0.33 & 1\\
-0.75 & 0 & -0.11 & 0.38 & 0.06 & 0.44 & 0.93 & -0.32\\
-0.49 & -0.67 & 0 & -0.07 & 0.26 & 0.99 & 0 & -0.94\\
0.46 & -0.59 & 0.84 & 0 & -0.13 & 0.33 & -0.98 & 0.96\\
0.25 & -0.26 & -0.6 & 0.77 & 0 & -0.92 & 0.65 & -0.58\\
-0.85 & 0.19 & 0.06 & -0.19 & -0.54 & 0 & 0.01 & -0.3 \\
0.52 & -0.01 & -0.55 & 0.26 & 0.44 & -0.26 & 0 & 0.08\\
-0.32 & -0.05 & 0.01 & -0.25 & 0.25 & -0.46 & 0.5 & 0
\end{pmatrix}
\begin{pmatrix}
1 \\ -1 \\ 1 \\ -1 \\ -1 \\ 1 \\ -1 \\ 0
\end{pmatrix}\\
=\frac12
\begin{pmatrix}
1 & -1 & 1 & -1 & -1 & 1 & -1 & 0
\end{pmatrix}
\begin{pmatrix}
2.05\\-1.79\\0.98\\3.33\\-2.43\\-0.26\\-0.98\\-1.22
\end{pmatrix}\\
=2.32
\end{align}
$$
### Calcul numérique

In [12]:
jastrow_parameters = [
0, -0.7, 0.28, -0.42, 0.31, 0.63, -0.33, 1,
-0.75, 0, -0.11, 0.38, 0.06, 0.44, 0.93, -0.32,
-0.49, -0.67, 0, -0.07, 0.26, 0.99, 0, -0.94,
0.46, -0.59, 0.84, 0, -0.13, 0.33, -0.98, 0.96,
0.25, -0.26, -0.6, 0.77, 0, -0.92, 0.65, -0.58,
-0.85, 0.19, 0.06, -0.19, -0.54, 0, 0.01, -0.3,
0.52, -0.01, -0.55, 0.26, 0.44, -0.26, 0, 0.08,
-0.32, -0.05, 0.01, -0.25, 0.25, -0.46, 0.5, 0
]
jastrow_coef = imp.jastrow_exponent(spin_up, spin_down, jastrow_parameters, NSITES)
print(jastrow_coef)

jastrow_coef = imp.jastrow_fastupdate(
    jastrow_coef,
    jastrow_parameters,
    spin_up,
    spin_down,
    new_spin_up,
    spin_down,
    NSITES,
    1,
    0
)
print(jastrow_coef)

0.92
4.01


## Calcul du Produit Scalaire $\braket{x|\phi_{\text{Pfaff}}}$

## Calcul de la Densité $\rho(x)$